# Next or Previous values

In [30]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [43]:
#Import Data
data = pd.read_csv("data.csv")
samp_sol = pd.read_csv("sample_submission.csv")
#print(data)

In [42]:
#Next method
data_next = data.copy()
data_next["x_e_out [-]"] = data_next["x_e_out [-]"].fillna(data_next["x_e_out [-]"].shift(-1))
next_sol = samp_sol.copy()
filtered_data_next = pd.merge(next_sol['id'], data_next, on='id', how='left')
filtered_data_next = filtered_data_next.dropna(subset=['x_e_out [-]'])
#print(filtered_data_next)
next_sol['x_e_out [-]'] = filtered_data_next['x_e_out [-]']
#print(next_sol)



#Previous method
data_previous = data.copy()
data_previous["x_e_out [-]"] = data_previous["x_e_out [-]"].fillna(data_previous["x_e_out [-]"].shift(1))
previous_sol = samp_sol.copy()
filtered_data_previous = pd.merge(previous_sol['id'], data_previous, on='id', how='left')
filtered_data_previous = filtered_data_previous.dropna(subset=['x_e_out [-]'])
#print(filtered_data_previous)
previous_sol['x_e_out [-]'] = filtered_data_previous['x_e_out [-]']
print(previous_sol)
#print(data_previous)

          id  x_e_out [-]
0          4      -0.0279
1          7       0.1203
2         10       0.1228
3         12       0.0515
4         23      -0.0124
...      ...          ...
10410  31633          NaN
10411  31634          NaN
10412  31637      -0.0195
10413  31640       0.0886
10414  31642      -0.1224

[10415 rows x 2 columns]


# K nearest Neighbors

In [41]:
# Create a copy of the original data dataframe
data_knn = data.copy()
# Specify the column containing the missing values
column_name = 'x_e_out [-]'
# Create an instance of the KNNImputer class
imputer = KNNImputer(n_neighbors=1)
# Perform KNN imputation on the specified column
data_knn[column_name] = imputer.fit_transform(data_knn[[column_name]])
# Print the data_knn dataframe
#print(data_knn)
knn_sol = samp_sol.copy()
filtered_data_knn = pd.merge(knn_sol['id'], data_knn, on='id', how='left')
filtered_data_knn = filtered_data_knn.dropna(subset=['x_e_out [-]'])
#print(filtered_data_knn)
knn_sol['x_e_out [-]'] = filtered_data_knn['x_e_out [-]']
#print(knn_sol)


# Maximum and Minimum Values

In [40]:
#Max Values
data_max = data.copy()
data_max['x_e_out [-]'] = data_max['x_e_out [-]'].fillna(data_max['x_e_out [-]'].max())
#print(data_max)
max_sol = samp_sol.copy()
filtered_data_max = pd.merge(max_sol['id'], data_max, on='id', how='left')
filtered_data_max = filtered_data_max.dropna(subset=['x_e_out [-]'])
#print(filtered_data_max)
max_sol['x_e_out [-]'] = filtered_data_max['x_e_out [-]']
#print(max_sol)

#Min Values
data_min = data.copy()
data_min['x_e_out [-]'] = data_min['x_e_out [-]'].fillna(data_min['x_e_out [-]'].min())
min_sol = samp_sol.copy()
filtered_data_min = pd.merge(min_sol['id'], data_min, on='id', how='left')
filtered_data_min = filtered_data_min.dropna(subset=['x_e_out [-]'])
#print(filtered_data_min)
min_sol['x_e_out [-]'] = filtered_data_min['x_e_out [-]']
#print(min_sol)




# Missing Value Prediction (Multivariable linear Regression)

In [39]:
data_reg = data.copy()
# Create the imputer model
imputer = IterativeImputer(estimator=LinearRegression())

# Perform imputation on the entire dataset
imputed_data = imputer.fit_transform(data_reg[['pressure [MPa]', 'mass_flux [kg/m2-s]', 'D_e [mm]', 'D_h [mm]', 'length [mm]', 'chf_exp [MW/m2]', 'x_e_out [-]']])

# Update the 'x_e_out [-]' column with imputed values
data_reg['x_e_out [-]'] = imputed_data[:, -1]
#print(data_reg)
reg_sol = samp_sol.copy()
filtered_data_reg = pd.merge(reg_sol['id'], data_reg, on='id', how='left')
filtered_data_reg = filtered_data_reg.dropna(subset=['x_e_out [-]'])
#print(filtered_data_reg)
reg_sol['x_e_out [-]'] = filtered_data_reg['x_e_out [-]']
#print(reg_sol)

# Missing Value Prediction (Linear Regression)

In [37]:
#This doesnt work yet
#data_linear = data.copy()
# Remove rows with missing values in any column
#data_clean = data_linear.dropna()
# Split the data into complete and missing subsets
#complete_data = data_clean.dropna(subset=['x_e_out [-]'])
#missing_data = data_clean[data_clean['x_e_out [-]'].isnull()]
# Create the regression model
#reg_model = LinearRegression()
# Fit the regression model
#reg_model.fit(complete_data[['pressure [MPa]', 'mass_flux [kg/m2-s]', 'D_e [mm]', 'D_h [mm]', 'length [mm]', 'chf_exp [MW/m2]']], complete_data['x_e_out [-]'])
# Predict the missing values
#predicted_values = reg_model.predict(missing_data[['pressure [MPa]', 'mass_flux [kg/m2-s]', 'D_e [mm]', 'D_h [mm]', 'length [mm]', 'chf_exp [MW/m2]']])
# Replace missing values with predicted values
#data_linear.loc[data['x_e_out [-]'].isnull(), 'x_e_out [-]'] = predicted_values

# Mean, Median, and Mode

In [38]:
#Mean Values
data_mean = data.copy()
data_mean['x_e_out [-]'] = data_mean['x_e_out [-]'].fillna(data_mean['x_e_out [-]'].mean())
mean_sol = samp_sol.copy()
filtered_data_mean = pd.merge(mean_sol['id'], data_mean, on='id', how='left')
filtered_data_mean = filtered_data_mean.dropna(subset=['x_e_out [-]'])
#print(filtered_data_mean)
mean_sol['x_e_out [-]'] = filtered_data_mean['x_e_out [-]']
#print(mean_sol)

#Mode Values
data_mode = data.copy()
mode_value = data_mode['x_e_out [-]'].mode()[0]
data_mode['x_e_out [-]'] = data_mode['x_e_out [-]'].fillna(mode_value)
mode_sol = samp_sol.copy()
filtered_data_mode = pd.merge(mode_sol['id'], data_mode, on='id', how='left')
filtered_data_mode = filtered_data_mode.dropna(subset=['x_e_out [-]'])
#print(filtered_data_mode)
mode_sol['x_e_out [-]'] = filtered_data_mode['x_e_out [-]']
#print(mode_sol)

#Median Values
data_median = data.copy()
median_value = data_median['x_e_out [-]'].median()
data_median['x_e_out [-]'] = data_median['x_e_out [-]'].fillna(median_value)
#print(data_median)
median_sol = samp_sol.copy()
filtered_data_median = pd.merge(median_sol['id'], data_median, on='id', how='left')
filtered_data_median = filtered_data_median.dropna(subset=['x_e_out [-]'])
#print(filtered_data_median)
median_sol['x_e_out [-]'] = filtered_data_median['x_e_out [-]']
#print(median_sol)


# Linear Interpolation

In [45]:
data_interpolate = data.copy()
data_interpolate['x_e_out [-]'] = data['x_e_out [-]'].interpolate()
#print(data_interpolate)
interpolate_sol = samp_sol.copy()
filtered_data_interpolate = pd.merge(interpolate_sol['id'], data_interpolate, on='id', how='left')
filtered_data_interpolate = filtered_data_interpolate.dropna(subset=['x_e_out [-]'])
#print(filtered_data_interpolate)
interpolate_sol['x_e_out [-]'] = filtered_data_interpolate['x_e_out [-]']
#print(interpolate_sol)